In [1]:
import os
import sys
import socket

import sys,uuid,datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [2]:
key = os.environ["MINIO_ACCESS_KEY"]
secret = os.environ["MINIO_SECRET_KEY"]
endpoint = os.environ["MINIO_SECRET_ENDPOINT"]
endpoint = "http://192.168.2.128:9000"
print(endpoint)

http://192.168.2.128:9000


In [3]:
#sc.stop()

spark = SparkSession.builder \
.master("k8s://https://kubernetes.docker.internal:6443") \
.appName("test_cluster_mode") \
.config("spark.hadoop.fs.s3a.access.key", key) \
.config("spark.hadoop.fs.s3a.secret.key", secret) \
.config("spark.hadoop.fs.s3a.endpoint", endpoint) \
.config("spark.hadoop.fs.s3a.path.style.access", "true") \
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0,org.apache.hadoop:hadoop-aws:3.2.0,com.amazonaws:aws-java-sdk-bundle:1.11.375") \
.config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
.config('spark.submit.deployMode', 'client') \
.config("spark.kubernetes.container.image", "spark:spark-docker") \
.config("spark.kubernetes.pyspark.pythonVersion", "3") \
.config("spark.kubernetes.authenticate.driver.serviceAccountName", "default") \
.config("spark.executor.instances", "1") \
.config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
.config("spark.kubernetes.executor.request.cores","0.5") \
.config("spark.kubernetes.executor.limit.cores","1") \
.config("jupyterService.jupyterPort", "30888") \
.config("serviceAccount", "spark") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.getOrCreate()

#.config("spark.driver.host", "10.1.2.104") \
#.config("spark.driver.port", "4040") \

sc = spark.sparkContext
#sc._conf.getAll()

In [6]:
data_frame = spark.read.json('s3a://real-estate/staging/201108_Solothurn_buy_0_flat.gz')
data_frame.show()

+--------------------+--------------------+
|     propertyDetails|            viewData|
+--------------------+--------------------+
|[334334, [Münsing...|[, https://statsp...|
|[102900, [Chur, S...|[, https://statsp...|
|[661265, [Deiting...|[, https://statsp...|
|[103561, [Evilard...|[[, 52908191, 201...|
|[414954, [Bern, S...|[, https://statsp...|
|[750219,, [, 2009...|[, https://statsp...|
+--------------------+--------------------+



In [5]:
spark.range(5).write.format("delta").mode("overwrite").save("s3a://test/delta/delta_jupyter_notebook12388")

In [6]:
df_d = spark.read.format("delta").load("s3a://test/delta/delta_jupyter_notebook12388")
df_d.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [7]:
sc.stop()